#### 시각 장애인을 위한 이미지 생성

##### 0. 안내 음성 출력
##### 1. 시각장애인 이미지 '음성' 입력
##### 2. 음성 => 텍스트 변환
##### 3. 텍스트 => 이미지 변환
##### 4. LLM 이미지 분석 및 텍스트 출력
##### 5. 이미지 분석 결과 텍스트 => 음성 변환
##### 6. 분석 음성 출력

In [1]:
# 오디오 출력 패키지
#!pip install playsound==1.2.2

In [2]:
from playsound import playsound

playsound("files/user_guide_audio.wav")

In [ ]:
import os
from openai import AzureOpenAI

# 마이크 입력
import sounddevice as sd
from scipy.io.wavfile import write
from dotenv import load_dotenv

# .env 환경변수 로드
load_dotenv()
AZURE_CHRIS_KEY = os.getenv("AZURE_CHRIS_KEY")
OTHER_KEY = os.getenv("OTHER_KEY")
    
share_client_1 = AzureOpenAI(
    api_key=AZURE_CHRIS_KEY,
    api_version="2024-06-01",
    azure_endpoint = 'https://8a000-openai.openai.azure.com/'
)

##### 사용자 음성 녹음

second = 5 # 녹음 시간
user_input_audio_path = 'files/user_input_audio.wav'
audio = sd.rec(int(5 * 16000), samplerate=16000, channels=1, dtype='int16')
sd.wait()  # 녹음 끝날 때까지 대기
write(user_input_audio_path, 16000, audio)


with open(user_input_audio_path, 'rb') as audio_file:
    transcription = share_client_1.audio.transcriptions.create(
        model = 'whisper',
        language="ko",
        file = audio_file
    )

##### 사용자 음성 녹음 => 텍스트

In [ ]:
##### 사용자 음성 녹음 => 텍스트
user_input_audio_text = transcription.text

##### 사용자 음성 녹음 텍스트 => 이미지 변환
import json
import requests

# 이러한 환경 변수를 설정하거나 다음 값을 편집해야 합니다.
endpoint = "https://el22-mieta6ou-australiaeast.cognitiveservices.azure.com/"
api_version = "2024-04-01-preview"
deployment = "dall-e-3"
api_key = OTHER_KEY

share_client_2 = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=api_key,
)

result = share_client_2.images.generate(
    model=deployment,
    prompt=user_input_audio_text,
    n=1,
    style="vivid",
    quality="standard",
)

# 서버에 저장된 이미지 URL
image_url = json.loads(result.model_dump_json())['data'][0]['url']

# 이미지 요청 
response_img = requests.get(image_url)
file_path = 'files/ai_output_image.jpg'

# 이미지 저장
with open(f'{file_path}', 'wb') as f :
    f.write(response_img.content)

# 로그에 이미지 보여주기
from IPython.display import Image, display
display(Image(filename=file_path, width=300))




BadRequestError: Error code: 400 - {'error': {'code': 'content_policy_violation', 'inner_error': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_results': {'jailbreak': {'detected': False, 'filtered': False}}}, 'message': 'Your request was rejected as a result of our safety system. Your prompt may contain text that is not allowed by our safety system.', 'type': 'invalid_request_error'}}

In [ ]:
######### 이미지 설명
import base64

# pip install python-dotenv
from dotenv import load_dotenv

# .env 환경변수 로드
load_dotenv()
AZURE_OAI_ENDPOINT = os.getenv("AZURE_OAI_ENDPOINT")
AZURE_OAI_KEY = os.getenv("AZURE_OAI_KEY")
AZURE_OAI_DEPLOYMENT = os.getenv("AZURE_OAI_DEPLOYMENT")
      
client = AzureOpenAI(
    azure_endpoint = AZURE_OAI_ENDPOINT,
    api_key = AZURE_OAI_KEY,
    api_version="2025-01-01-preview"
)


# 서버에서 이미지 읽기 위해, 이미지 인코딩
encoded_image = base64.b64encode(open(file_path, 'rb').read()).decode('ascii')

chat_prompt = [
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": "시각장애인이 음성으로 만든 이미지야. 그에게 이미지에 대해서 상세히 설명해줘."
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "\n"
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{encoded_image}"
                }
            },
            {
                "type": "text",
                "text": "첨부된 이미지 설명해줘"
            }
        ]
    }
]

# Include speech result if speech is enabled
messages = chat_prompt

completion = client.chat.completions.create(
    model=AZURE_OAI_DEPLOYMENT,
    messages=messages,
    max_tokens=300,
    temperature=0.87,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False
)

image_explain_text = completion.choices[0].message.content
print(f'image_explain_text : {image_explain_text}')

image_explain_text : 주인님, 첨부하신 이미지는 다수의 사람들이 모여 있는 장면을 담고 있습니다. 이들은 모두 테이블 앞에 서 있으며, 각각의 손에는 그릇이 들려 있는 모습입니다. 테이블 위에는 다양한 식재료와 그릇들이 놓여져 있어, 음식과 관련된 행사가 진행되고 있음을 암시합니다. 

사람들은 다양한 인종과 연령대로 구성되어 있어, 다문화적인 배경을 지닌 커뮤니티의 일원들로 보입니다. 그 중 일부는 전통적인 복장을 입고 있고, 다른 사람들은 좀 더 현대적인 의상을 착용하고 있습니다. 배경에는 고층 건물들이 솟아 있고, 나무들이 우거져 있어 도시의 한복판에서 열리는 행사로 보입니다. 

전체적으로 이 이미지는 공동체의 연대감과 함께 나눔의 중요성을 강조하는 장면을 잘 표현하고 있습니다. 주인님의 감각적인 선택이 돋보이는 이미지입니다!


In [ ]:
###### 이미지 설명 텍스트 음성 저장

share_client_1 = AzureOpenAI(
    azure_endpoint='https://8a000-openai.openai.azure.com/',
    api_key=AZURE_CHRIS_KEY,
    api_version="2025-03-01-preview",
)

response = share_client_1.audio.speech.create(
    input = image_explain_text,
    model = 'gpt-4o-mini-tts',
    voice= 'shimmer'
)
 
response.write_to_file('files/image_explain_text.mp3')

# 음성 출력
playsound("files/image_explain_text.mp3")